<a href="https://colab.research.google.com/github/Lucyy07/LSTM-learning/blob/main/test_lstm_modelOptimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas_datareader.data as web
import datetime
start=datetime.datetime(2000,1,1)
end=datetime.datetime(2021,9,1)
df=web.DataReader('GOOGL','stooq',start,end)

In [2]:
# 按日期排序
df.dropna(inplace=True)  # 一定要先删除空值，否则神经网络计算不出损失值
df.sort_index(inplace=True)
print(df)

                Open      High       Low     Close      Volume
Date                                                          
2004-08-19    50.000    52.030    47.980    50.170  44703800.0
2004-08-20    50.505    54.540    50.250    54.155  22857200.0
2004-08-23    55.375    56.740    54.525    54.700  18274400.0
2004-08-24    55.620    55.800    51.785    52.435  15262600.0
2004-08-25    52.480    54.000    51.940    53.000   9197800.0
...              ...       ...       ...       ...         ...
2021-08-26  2835.000  2848.350  2827.140  2828.810   1030466.0
2021-08-27  2833.050  2890.250  2829.940  2880.080   1439010.0
2021-08-30  2888.000  2919.410  2883.260  2891.810   1221710.0
2021-08-31  2902.940  2903.425  2885.620  2893.950   1122438.0
2021-09-01  2900.000  2925.075  2897.670  2904.310   1096805.0

[4289 rows x 5 columns]


In [4]:
# 创建要预测的标签
# 预测时间
# 删除，因为设置成了参数 pre_days=10
def LSTM_data_precessing(df, mem_his_days, pre_days):
  df['label']=df['Close'].shift(-pre_days)
  print(df)
  # 数据预处理：转换特征值
  from sklearn.preprocessing import StandardScaler
  scaler=StandardScaler()
  scal_X=scaler.fit_transform(df.iloc[:,:-1])
  #print(scal_X)
  # 转换成以mem_his_days天为一个时间列的数据
  # 删除此句，因为改成了参数  mem_his_days=10
  from collections import deque
  deq=deque(maxlen=mem_his_days)

  X=[]
  for i in scal_X:
    deq.append(list(i))
    if len(deq)==mem_his_days:
      X.append(list(deq))

  X_lately=X[-pre_days:]
  X=X[:-pre_days]
  #print(len(X))
  #print(len(X_lately))
  # 打包处理目标预测的值
  y=df['label'].values[mem_his_days-1:-pre_days]
  #print(len(y))
  # 转换成numpy的数组类型
  import numpy as np
  x=np.array(X)
  y=np.array(y)
  #print(x.shape)
  #print(y.shape)
  return x,y,X_lately

In [6]:
x,y,X_lately=LSTM_data_precessing(df, 5, 10)
print(x.shape)
print(x[0])

                Open      High       Low     Close      Volume   label
Date                                                                  
2004-08-19    50.000    52.030    47.980    50.170  44703800.0  50.755
2004-08-20    50.505    54.540    50.250    54.155  22857200.0  50.005
2004-08-23    55.375    56.740    54.525    54.700  18274400.0  50.790
2004-08-24    55.620    55.800    51.785    52.435  15262600.0  51.150
2004-08-25    52.480    54.000    51.940    53.000   9197800.0  51.155
...              ...       ...       ...       ...         ...     ...
2021-08-26  2835.000  2848.350  2827.140  2828.810   1030466.0     NaN
2021-08-27  2833.050  2890.250  2829.940  2880.080   1439010.0     NaN
2021-08-30  2888.000  2919.410  2883.260  2891.810   1221710.0     NaN
2021-08-31  2902.940  2903.425  2885.620  2893.950   1122438.0     NaN
2021-09-01  2900.000  2925.075  2897.670  2904.310   1096805.0     NaN

[4289 rows x 6 columns]
[[-1.08801034 -1.08523099 -1.08961219 -1.0866679   4

In [1]:
# 使用循环多次测试最佳参数
pre_days=10
mem_days=[5,10,15]
lstm_layers=[1,2,3]
units=[16,32]
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
for the_mem_days in mem_days:
  for the_lstm_layers in lstm_layers:
    for the_units in units:
      filePath='{val_mape:.2f}_epochs_40_'+f'mem_{the_mem_days}_lstm{the_lstm_layers}_units_{the_units}'
      checkPoint=ModelCheckpoint(filePath=filePath, save_weights_only=False,monitor='val_mape',mode='min',save_best_only=True)
      x,y,X_lately=LSTM_data_precessing(df, the_mem_days, pre_days)
      x_train, x_test, y_train, y_test=train_test_split(x,y,shuffle=False,test_size=0.1)
      model=Sequential()
      model.add(LSTM(the_units, input_shape=x.shape[1:],activation='relu',return_sequences=True))   # 此处input_shape只是指明输入的形式，并不是指输入数据
      model.add(Dropout(0.1))

      for i in range(the_lstm_layers):
        model.add(LSTM(the_units,activation='relu',return_sequences=True))
        model.add(Dropout(0.1))

      # model.add(LSTM(10,activation='relu'))
      # model.add(Dropout(0.1))

      model.add(Dense(the_units, activation='relu'))
      model.add(Dropout(0.1))

      model.add(Dense(1))

      model.compile(optimizer='adam',loss='mse',metrics=['mape'])

      model.fit(x_train,y_train,batch_size=32,epochs=60,validation_data=(x_test,y_test))
      print(filePath)

{val_mape:.2f}_40_mem_5_lstm1_units_16
{val_mape:.2f}_40_mem_5_lstm1_units_32
{val_mape:.2f}_40_mem_5_lstm2_units_16
{val_mape:.2f}_40_mem_5_lstm2_units_32
{val_mape:.2f}_40_mem_5_lstm3_units_16
{val_mape:.2f}_40_mem_5_lstm3_units_32
{val_mape:.2f}_40_mem_10_lstm1_units_16
{val_mape:.2f}_40_mem_10_lstm1_units_32
{val_mape:.2f}_40_mem_10_lstm2_units_16
{val_mape:.2f}_40_mem_10_lstm2_units_32
{val_mape:.2f}_40_mem_10_lstm3_units_16
{val_mape:.2f}_40_mem_10_lstm3_units_32
{val_mape:.2f}_40_mem_15_lstm1_units_16
{val_mape:.2f}_40_mem_15_lstm1_units_32
{val_mape:.2f}_40_mem_15_lstm2_units_16
{val_mape:.2f}_40_mem_15_lstm2_units_32
{val_mape:.2f}_40_mem_15_lstm3_units_16
{val_mape:.2f}_40_mem_15_lstm3_units_32


In [ ]:
# 划分训练集和测试集


3843


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM,Dense,Dropout
# model=Sequential()
# model.add(LSTM(10, input_shape=x.shape[1:],activation='relu',return_sequences=True))   # 此处input_shape只是指明输入的形式，并不是指输入数据
# model.add(Dropout(0.1))

# model.add(LSTM(10,activation='relu',return_sequences=True))
# model.add(Dropout(0.1))

# model.add(LSTM(10,activation='relu'))
# model.add(Dropout(0.1))

# model.add(Dense(10, activation='relu'))
# model.add(Dropout(0.1))

# model.add(Dense(1))

# model.compile(optimizer='adam',loss='mse',metrics=['mape'])

# model.fit(x_train,y_train,batch_size=32,epochs=60,validation_data=(x_test,y_test))

Epoch 1/60
121/121 [==============================] - 7s 21ms/step - loss: 455391.1875 - mape: 84.6283 - val_loss: 124986.2891 - val_mape: 44.8779
Epoch 2/60
121/121 [==============================] - 2s 16ms/step - loss: 72153.1797 - mape: 33.8705 - val_loss: 17620.7637 - val_mape: 22.6395
Epoch 3/60
121/121 [==============================] - 2s 16ms/step - loss: 53948.7305 - mape: 26.8533 - val_loss: 10271.9844 - val_mape: 16.7014
Epoch 4/60
121/121 [==============================] - 2s 16ms/step - loss: 43130.2461 - mape: 23.7753 - val_loss: 26358.3555 - val_mape: 18.3589
Epoch 5/60
121/121 [==============================] - 2s 16ms/step - loss: 42321.1250 - mape: 22.7552 - val_loss: 13123.5127 - val_mape: 16.3967
Epoch 6/60
121/121 [==============================] - 2s 16ms/step - loss: 34005.4414 - mape: 21.4242 - val_loss: 11368.4414 - val_mape: 13.0096
Epoch 7/60
121/121 [==============================] - 2s 16ms/step - loss: 35702.6328 - mape: 21.1776 - val_loss: 8769.4121 - va